In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing 
from category_encoders import *
from sklearn.preprocessing import LabelEncoder
%matplotlib inline

from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, f1_score, precision_score
from sklearn.model_selection import RandomizedSearchCV

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn import datasets, linear_model, metrics
from sklearn.metrics import  confusion_matrix
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.datasets import make_classification
import warnings
warnings.filterwarnings("ignore")

In [19]:
data = pd.read_csv("./RNA_seq.csv")
dataT=np.array(data)
data=dataT.T
co=data[0]
data1=np.delete(data,0,axis=0)
data=data1
datadf= pd.DataFrame(data=data[0:,0:],columns=co)
datadf.head()
data=datadf
data= data.replace("NOTLC",value=0)
data= data.replace("LC",value=1)
X=data.drop(['Group'],axis=1)
y=data['Group']

In [20]:
column=X.columns
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X=sc.fit_transform(X)

In [21]:
X=pd.DataFrame(data=X,columns=column)

In [22]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42,stratify=None)
print(f'Train shape : {X_train.shape}\nTest shape: {X_test.shape}')

Train shape : (878, 60660)
Test shape: (220, 60660)


In [23]:
def crosspredict(estimator,Xtrain,ytrain,cv):
    print("cross-validate across the entire data set")
    y_pred_cross=cross_val_predict(estimator,Xtrain,ytrain,cv=cv)
    confusion_cross=confusion_matrix(ytrain,y_pred_cross)
    a=accuracy_score(ytrain,y_pred_cross)
    p=precision_score(ytrain, y_pred_cross)
    r=recall_score(ytrain, y_pred_cross)
    f1=f1_score(ytrain, y_pred_cross)
    wf1=f1_score(ytrain, y_pred_cross, average='weighted')
    #auc=roc_auc_score(ytrain,estimator.predict_proba(Xtrain)[:,1])
    print('the confusion_matrix of the model is : \n',confusion_cross)
    print('the accuracy of the model is : ',a)
    print("the precision score of the model is : ", p)
    print("the recall score of the model is :", r)
    print('the f1_score of the model  is :',f1)
    print('the weighted_f1 of the model is :',wf1)
    print('the classification_report is :\n',classification_report(ytrain, y_pred_cross,digits=4))
    #print('the auc is :',auc)
    return a,p,r,f1,wf1

In [12]:
from sklearn.base import ClassifierMixin
from sklearn.base import TransformerMixin, BaseEstimator
class MyEstimator(BaseEstimator, ClassifierMixin):
    def __init__(self, model, mask) -> None:
        super().__init__()
        self.mask = mask
        self.model = model
    
    def fit(self, X, y=None):
        self.model.fit(X[self.mask],y)
        return self

    def predict(self, X):
        return self.model.predict(X[self.mask])
    
    def predict_proba(self,X):
        return self.model.predict_proba(X[self.mask])

In [40]:
from imblearn.over_sampling import SMOTE
over_samples = SMOTE(random_state =42)
X_smote, y_smote= over_samples.fit_sample(X, y)
X_smote=pd.DataFrame(data=X_smote,columns=column)

In [59]:
from itertools import *
sel_seq_dt_mask=['ENSG00000003147.19','ENSG00000004455.17','ENSG00000006327.14','ENSG00000018510.17','ENSG00000067064.11','ENSG00000154813.10','ENSG00000204305.14','ENSG00000234425.1','ENSG00000262772.2','ENSG00000272477.1']
sel_seq_rf_mask=['ENSG00000106089.12', 'ENSG00000204305.14', 'ENSG00000223982.3','ENSG00000229693.2']
sel_seq_ada_mask=['ENSG00000000005.6', 'ENSG00000002933.9','ENSG00000019144.19', 'ENSG00000070366.14','ENSG00000091262.16','ENSG00000106089.12','ENSG00000204305.14','ENSG00000233117.4']
sel_seq_gb_mask=['ENSG00000000005.6','ENSG00000000971.16', 'ENSG00000001631.16','ENSG00000002587.10', 'ENSG00000004779.10','ENSG00000106089.12', 'ENSG00000144130.11', 'ENSG00000204305.14']
sel_seq_xgb_mask=['ENSG00000106089.12','ENSG00000170989.10', 'ENSG00000204305.14']
sel_seq_KN_mask=['ENSG00000012983.11','ENSG00000106089.12','ENSG00000158578.21','ENSG00000204305.14','ENSG00000214796.8', 'ENSG00000277878.1']
sel_seq_Lightgbm_mask=['ENSG00000002834.18','ENSG00000106089.12', 'ENSG00000170989.10','ENSG00000204305.14']
sel_seq_SVC_mask=['ENSG00000002726.21','ENSG00000039068.19','ENSG00000120279.6','ENSG00000158578.21','ENSG00000182685.7','ENSG00000204305.14']
sel_seq_GNB_mask=['ENSG00000011052.21', 'ENSG00000052841.15','ENSG00000062194.16', 'ENSG00000068001.14', 'ENSG00000102871.16','ENSG00000124532.15', 'ENSG00000204305.14', 'ENSG00000227066.2']
sel_seq_lr_mask=['ENSG00000102547.19','ENSG00000135604.10','ENSG00000159352.16','ENSG00000224215.1','ENSG00000234481.2','ENSG00000252275.1','ENSG00000271555.1']

feature=list(set(sel_seq_dt_mask+sel_seq_rf_mask+sel_seq_ada_mask+sel_seq_xgb_mask+sel_seq_GNB_mask))

model_dt_fs = MyEstimator(DecisionTreeClassifier(), sel_seq_dt_mask)
model_rf_fs = MyEstimator(RandomForestClassifier(), sel_seq_rf_mask)
model_ada_fs = MyEstimator(AdaBoostClassifier(), sel_seq_ada_mask)
model_gb_fs = MyEstimator(GradientBoostingClassifier(), sel_seq_gb_mask)
model_xgb_fs = MyEstimator(XGBClassifier(), mask=sel_seq_xgb_mask)
model_KN_fs = MyEstimator(KNeighborsClassifier(), mask=sel_seq_KN_mask)
model_Lightgbm_fs = MyEstimator(lgb.LGBMClassifier(), mask=sel_seq_Lightgbm_mask)
model_SVC_fs = MyEstimator(SVC(),mask=sel_seq_SVC_mask)
model_GNB_fs = MyEstimator(GaussianNB(),mask=sel_seq_GNB_mask)
model_lr_fs = MyEstimator(LogisticRegression(),mask=sel_seq_lr_mask)

from sklearn.ensemble import VotingClassifier
model_voting_cross = VotingClassifier([  
    ("dt",model_dt_fs),
    ("rf",model_rf_fs ),
    ("ada", model_ada_fs),
    ("xgb", model_xgb_fs),
    ("GNB",model_GNB_fs),
], voting="hard")
a,b,c,d,e=crosspredict(model_voting_cross,X_smote,y_smote,10)

cross-validate across the entire data set
the confusion_matrix of the model is : 
 [[994   0]
 [  1 993]]
the accuracy of the model is :  0.9994969818913481
the precision score of the model is :  1.0
the recall score of the model is : 0.9989939637826962
the f1_score of the model  is : 0.9994967287367891
the weighted_f1 of the model is : 0.9994969817640708
the classification_report is :
               precision    recall  f1-score   support

           0     0.9990    1.0000    0.9995       994
           1     1.0000    0.9990    0.9995       994

    accuracy                         0.9995      1988
   macro avg     0.9995    0.9995    0.9995      1988
weighted avg     0.9995    0.9995    0.9995      1988



In [57]:
from mlxtend.classifier import StackingClassifier
model_Stacking_cross = StackingClassifier(classifiers=[
    (model_dt_fs),
    (model_rf_fs ),
    (model_ada_fs),
    (model_xgb_fs),
    (model_GNB_fs),
],meta_classifier=LogisticRegression(C=0.1))

a,b,c,d,e=crosspredict(model_Stacking_cross,X_smote,y_smote,10)

cross-validate across the entire data set
the confusion_matrix of the model is : 
 [[994   0]
 [  1 993]]
the accuracy of the model is :  0.9994969818913481
the precision score of the model is :  1.0
the recall score of the model is : 0.9989939637826962
the f1_score of the model  is : 0.9994967287367891
the weighted_f1 of the model is : 0.9994969817640708
the classification_report is :
               precision    recall  f1-score   support

           0     0.9990    1.0000    0.9995       994
           1     1.0000    0.9990    0.9995       994

    accuracy                         0.9995      1988
   macro avg     0.9995    0.9995    0.9995      1988
weighted avg     0.9995    0.9995    0.9995      1988



In [54]:
models=[model_dt_fs,model_rf_fs,model_ada_fs,model_gb_fs,model_xgb_fs,model_KN_fs,model_Lightgbm_fs,model_SVC_fs,model_GNB_fs,model_lr_fs]

In [56]:
for i in models:
    print("the model is:",i)
    a,b,c,d,e=crosspredict(i,X_smote,y_smote,10)

the model is: MyEstimator(mask=['ENSG00000003147.19', 'ENSG00000004455.17',
                  'ENSG00000006327.14', 'ENSG00000018510.17',
                  'ENSG00000067064.11', 'ENSG00000154813.10',
                  'ENSG00000204305.14', 'ENSG00000234425.1',
                  'ENSG00000262772.2', 'ENSG00000272477.1'],
            model=DecisionTreeClassifier())
cross-validate across the entire data set
the confusion_matrix of the model is : 
 [[989   5]
 [  6 988]]
the accuracy of the model is :  0.994466800804829
the precision score of the model is :  0.9949647532729103
the recall score of the model is : 0.993963782696177
the f1_score of the model  is : 0.9944640161046804
the weighted_f1 of the model is : 0.9944667994047786
the classification_report is :
               precision    recall  f1-score   support

           0     0.9940    0.9950    0.9945       994
           1     0.9950    0.9940    0.9945       994

    accuracy                         0.9945      1988
   macro avg 

the confusion_matrix of the model is : 
 [[994   0]
 [234 760]]
the accuracy of the model is :  0.8822937625754527
the precision score of the model is :  1.0
the recall score of the model is : 0.7645875251509054
the f1_score of the model  is : 0.8665906499429874
the weighted_f1 of the model is : 0.8806400594449411
the classification_report is :
               precision    recall  f1-score   support

           0     0.8094    1.0000    0.8947       994
           1     1.0000    0.7646    0.8666       994

    accuracy                         0.8823      1988
   macro avg     0.9047    0.8823    0.8806      1988
weighted avg     0.9047    0.8823    0.8806      1988

the model is: MyEstimator(mask=['ENSG00000102547.19', 'ENSG00000135604.10',
                  'ENSG00000159352.16', 'ENSG00000224215.1',
                  'ENSG00000234481.2', 'ENSG00000252275.1',
                  'ENSG00000271555.1'],
            model=LogisticRegression())
cross-validate across the entire data set
the co